# Sim-launcher
This script show how to launch sims using Python

### 1. Package imports

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from pathlib import Path  # Python 3.6+ only
import os
import psycopg2
from psycopg2.extras import execute_values
import random
import time

### 2. Environment Variables

In [7]:
# Load the environment variables
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)
# Print this to see if the env variables are read now
os.getenv("COMPOSE_PROJECT_NAME")

'evidss'

### 3. Database connection (writer)

In [8]:
# Generic function to test the connection to the database
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(    
            host=os.getenv("MAIN_HOST"),
            database=os.getenv("MAIN_DB"),
            user=os.getenv("DBWRITE_USER"),
            password=os.getenv("DBWRITE_PWD"), 
            port = os.getenv("MAIN_PORT")
        )
		
        # create a cursor
        cur = conn.cursor()
        
	# execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [9]:
# Make the test database connection
connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.5 (Ubuntu 12.5-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit',)
Database connection closed.


In [10]:
conn = psycopg2.connect(    
    host=os.getenv("MAIN_HOST"),
    database=os.getenv("MAIN_DB"),
    user=os.getenv("DBWRITE_USER"),
    password=os.getenv("DBWRITE_PWD"), 
    port = os.getenv("MAIN_PORT")
)

# create a cursor
cur = conn.cursor()

### 4. Database queries

#### 4.1 Base-case analysis

In [8]:
sql_set = 'INSERT INTO analysis_sets (description) VALUES (%s);'
set_data = 'first set as test'

In [9]:
sql_analysis = 'INSERT INTO analysis_record (user_id, status, include_tesla) VALUES (%s, %s, %s);'
analysis_data = (os.getenv("AUTH0_USERID"), 'inserted', 'FALSE')

In [10]:
sql_user = 'INSERT INTO user_details (user_id, user_name, email_id) VALUES (%s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET last_submit_date = NOW();'
user_data = (os.getenv("AUTH0_USERID"), os.getenv("AUTH0_USERNAME"), os.getenv("AUTH0_EMAIL"))

In [11]:
sql_params = 'INSERT INTO analysis_params (param_id, param_value) VALUES %s'
params_data = [(1, '123' ),(2, '70' ),(14, '10' ),(3, '80' ),(4, '100' ),(9, '40' ),(10, '50' ),(11, '25' ),(12, '23' ),(13, '20' ), (15, '1' ), ( 16, '10' ), (17, '80' ), (18, '0' ), (19, '60' ), (20, '20' ), (21, '200' )]

##### Launch an analysis

In [14]:
# cur.mogrify(sql_set, (set_data,))

In [11]:
# All these should be executed together as a transaction
################### the following will launch a sim 
cur.execute(sql_set, (set_data, ))
cur.execute(sql_analysis,  analysis_data)
cur.execute(sql_user,  user_data)
execute_values(cur, sql_params, params_data)
conn.commit()

Launch a set of analysis requests

In [25]:
################### The following will launch 5 sims with varying seed 
#######################################################################
create_new_set = True # a boolean to encode whether to create a new set for this analysis request or add this to the previous one 
number_of_sims = 5 # launch five sims

for i in range(0, number_of_sims): 
    seed = random.randint(1, 1000)
    if(create_new_set):
        set_data = 'Testing the effect of varying seed with everything else constant'
        cur.execute(sql_set, (set_data, ))
    cur.execute(sql_analysis,  analysis_data)
    cur.execute(sql_user,  user_data)
    # change the seed 
    params_data.pop(0) # remove the current list element for parameter 'global_seed' (param_id = 1)
    params_data.insert(0, (1, str(seed)))
    execute_values(cur, sql_params, params_data)
    create_new_set = False # since the next 4 simulations belong to the same set
    conn.commit()

Launch a set of analysis requests with different critical distance

In [12]:
create_new_set = True # a boolean to encode whether to create a new set for this analysis request or add this to the previous one 
critical_dists = [50, 70, 90] # critical distance set

for dist in critical_dists: 
    if(create_new_set):
        set_data = 'Testing the effect of varying critical distance keeping everything else constant'
        cur.execute(sql_set, (set_data, ))
    cur.execute(sql_analysis,  analysis_data)
    cur.execute(sql_user,  user_data)
    # change the seed 
    params_data.pop(1) # remove the current list element for parameter 'critical_distance' (param_id = 2), list pos 1
    params_data.insert(1, (2, str(dist)))
    execute_values(cur, sql_params, params_data)
    create_new_set = False # since the next 4 simulations belong to the same set
    conn.commit()
    time.sleep(3) # sleep for 3 seconds so the next analysis request is 3 seconds later

#### 4.2 Adding new chargers

In [24]:
# The order of columns in the csv is important
new_evse_scenario = pd.read_csv('new_evse_scenario.csv')
new_evse_scenario

,latitude,longitude,dcfc_plug_count,dcfc_power,level2_plug_count,level2_power,dcfc_fixed_charging_price,dcfc_var_charging_price_unit,dcfc_var_charging_price,dcfc_fixed_parking_price,dcfc_var_parking_price_unit,dcfc_var_parking_price,level2_fixed_charging_price,level2_var_charging_price_unit,level2_var_charging_price,level2_fixed_parking_price,level2_var_parking_price_unit,level2_var_parking_price,connector_code
0,48.370848,-118.768157,1,50,1,10,0.5,min,0.5,0.5,min,0.5,0.5,min,0.5,0.5,min,0.5,3


In [25]:
new_evse_scenario.dtypes

latitude                           float64
 longitude                         float64
 dcfc_plug_count                     int64
 dcfc_power                          int64
 level2_plug_count                   int64
 level2_power                        int64
 dcfc_fixed_charging_price         float64
 dcfc_var_charging_price_unit       object
 dcfc_var_charging_price           float64
 dcfc_fixed_parking_price          float64
 dcfc_var_parking_price_unit        object
 dcfc_var_parking_price            float64
 level2_fixed_charging_price       float64
 level2_var_charging_price_unit     object
 level2_var_charging_price         float64
 level2_fixed_parking_price        float64
 level2_var_parking_price_unit      object
 level2_var_parking_price          float64
 connector_code                      int64
dtype: object

In [34]:
new_evse_data = [tuple(row) for row in new_evse_scenario.itertuples(index=False)] 

In [35]:
sql_new_evse = """INSERT INTO new_evses (latitude, longitude,
                dcfc_plug_count, dcfc_power, level2_plug_count, level2_power,
                dcfc_fixed_charging_price, dcfc_var_charging_price_unit,
                dcfc_var_charging_price, dcfc_fixed_parking_price, dcfc_var_parking_price_unit,
                dcfc_var_parking_price, level2_fixed_charging_price, level2_var_charging_price_unit,
                level2_var_charging_price, level2_fixed_parking_price, level2_var_parking_price_unit,
                level2_var_parking_price, connector_code) VALUES %s"""

In [38]:
# All these should be executed together as a transaction
set_data = 'testing a new evse scenario using code'
cur.execute(sql_set, (set_data, ))
cur.execute(sql_analysis,  analysis_data)
cur.execute(sql_user,  user_data)
execute_values(cur, sql_params, params_data)
if (len(new_evse_scenario.index) > 0):
    execute_values(cur, sql_new_evse, new_evse_data)
# use execute values with new evses by creating an array of tuples 
conn.commit()

_____________________
______________________

### 4.3 Disabling chargers

In [16]:
seeds = [123, 366, 495]
disabled_charger_count = 10

In [25]:
sql_bevse_id = """select bevse_id from built_evse where dcfc_count >= 1 and in_service = 'true' and connector_code IN (1, 2, 3);"""
bevse_ids = pd.read_sql_query(sql=sql_bevse_id, con=conn, params=())
bevse_ids

,bevse_id
0,39778.0
1,39780.0
2,39788.0
3,39789.0
4,39791.0
...,...
136,167523.0
137,167562.0
138,167758.0
139,168082.0


In [34]:
sql_set = 'INSERT INTO analysis_sets (description) VALUES (%s);'

In [35]:
sql_analysis = 'INSERT INTO analysis_record (user_id, status, include_tesla) VALUES (%s, %s, %s);'
analysis_data = (os.getenv("AUTH0_USERID"), 'inserted', 'FALSE')

In [36]:
sql_user = 'INSERT INTO user_details (user_id, user_name, email_id) VALUES (%s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET last_submit_date = NOW();'
user_data = (os.getenv("AUTH0_USERID"), os.getenv("AUTH0_USERNAME"), os.getenv("AUTH0_EMAIL"))

In [37]:
sql_params = 'INSERT INTO analysis_params (param_id, param_value) VALUES %s'
params_data = [(1, '123' ),(2, '70' ),(14, '10' ),(3, '80' ),(4, '100' ),(9, '40' ),(10, '50' ),(11, '25' ),(12, '23' ),(13, '20' ), (15, '1' ), ( 16, '10' ), (17, '80' ), (18, '0' ), (19, '60' ), (20, '20' ), (21, '200' )]

In [55]:
sql_bevse_subset = '''select bevse_id, longitude, latitude, connector_code, dcfc_count, dcfc_fixed_charging_price, dcfc_var_charging_price_unit, dcfc_var_charging_price, dcfc_fixed_parking_price, dcfc_var_parking_price_unit, dcfc_var_parking_price from built_evse where dcfc_count >= 1 and in_service = 'true' and connector_code IN (1, 2, 3) and bevse_id NOT IN %s '''

In [ ]:
sql_evses_now = 'INSERT INTO evses_now (evse_id, latitude, longitude, dcfc_plug_count)'

In [59]:
################### The following will launch 3 sims with varying disabled chargers 
#######################################################################
create_new_set = True # a boolean to encode whether to create a new set for this analysis request or add this to the previous one 
number_of_sims = len(seeds) # launch five sims

for i in range(0, number_of_sims): 
    seed = seeds[i]
    disable_bevse_ids = []
    rints = random.sample(range(len(bevse_ids)), disabled_charger_count)
    print(rints)
    disable_bevse_ids = bevse_ids['bevse_id'].take(rints)
    # for j in range(0, disabled_charger_count):
    #    rint = random.randint(0, len(bevse_ids) - 1)
    #    rbevse_id = bevse_ids['bevse_id'][rint]
    #    disable_bevse_ids.append(rbevse_id)
    print(disable_bevse_ids.tolist())
    # get evses that have not been disabled
    bevses_subset = pd.read_sql_query(sql=sql_bevse_subset, con=conn, params=(tuple(disable_bevse_ids.tolist()),))
    # Rename columns and add 'b' to id
    bevses_subset.rename(columns={"bevse_id": "evse_id"}, inplace=True)
    bevses_subset['evse_id'] = 'b' + bevses_subset['evse_id'].astype(str)
    print(bevses_subset)
    for index, row in bevses_subset.iterrows():
        
    if(create_new_set):
        set_data = 'Testing the effect of disabling chargers'
        # cur.execute(sql_set, (set_data, ))
    # cur.execute(sql_analysis,  analysis_data)
    # cur.execute(sql_user,  user_data)
    # change the seed 
    params_data.pop(0) # remove the current list element for parameter 'global_seed' (param_id = 1)
    params_data.insert(0, (1, str(seed)))
    # execute_values(cur, sql_params, params_data)
    
    create_new_set = False # since the next 4 simulations belong to the same set
    # conn.commit()

[44, 2, 10, 65, 6, 96, 31, 95, 37, 122]
[93550.0, 39788.0, 65859.0, 99282.0, 61030.0, 143215.0, 92487.0, 143212.0, 93339.0, 154419.0]
       evse_id   longitude   latitude  connector_code  dcfc_count  \
0     b39778.0 -122.454132  48.755607             1.0         1.0   
1     b39780.0 -122.333257  47.458466             1.0         1.0   
2     b39789.0 -122.294997  47.207367             1.0         1.0   
3     b39791.0 -122.218800  47.448354             1.0         1.0   
4     b39796.0 -117.312518  47.657264             1.0         1.0   
..         ...         ...        ...             ...         ...   
126  b167523.0 -118.861160  46.659027             3.0         2.0   
127  b167562.0 -122.104565  47.671794             3.0         3.0   
128  b167758.0 -122.297569  47.155976             3.0         4.0   
129  b168082.0 -122.151151  47.754220             3.0         4.0   
130  b168088.0 -122.512304  48.789282             3.0         4.0   

     dcfc_fixed_charging_price dcfc_v